In [53]:
import pymongo

from time import mktime
from dateutil import parser
from time import strptime
from pymongo import MongoClient

Make a connection

In [39]:
client = MongoClient(host='192.168.43.225')
db = client.soccerdb

In [41]:
db.matches.find_one()

{'_id': ObjectId('5d9dd62d1c801fea8e2419b7'),
 'competitionId': 364,
 'date': 'May 13, 2018 at 4:00:00 PM GMT+2',
 'dateutc': '2018-05-13 14:00:00',
 'duration': 'Regular',
 'gameweek': 38,
 'label': 'Burnley - AFC Bournemouth, 1 - 2',
 'referees': [{'refereeId': 385705, 'role': 'referee'},
  {'refereeId': 385733, 'role': 'firstAssistant'},
  {'refereeId': 385922, 'role': 'secondAssistant'},
  {'refereeId': 388333, 'role': 'fourthOfficial'}],
 'roundId': 4405654,
 'seasonId': 181150,
 'status': 'Played',
 'teamsData': {'1646': {'coachId': 8880,
   'formation': {'bench': [{'goals': 'null',
      'ownGoals': '0',
      'playerId': 77502,
      'redCards': '0',
      'yellowCards': '0'},
     {'goals': 'null',
      'ownGoals': '0',
      'playerId': 270828,
      'redCards': '0',
      'yellowCards': '0'},
     {'goals': 'null',
      'ownGoals': '0',
      'playerId': 9164,
      'redCards': '0',
      'yellowCards': '0'},
     {'goals': 'null',
      'ownGoals': '0',
      'playerId': 

### Add timestamp field to matches documents

In [58]:
for doc in db.matches.find({}, {'wyId': 1, 'date': 1}):
    match_id = doc['wyId']
    date = doc['date']
    date = int(mktime(parser.parse(date).timetuple()))
    db.matches.update_one({'wyId': match_id}, {'$set': {'timestamp': date}})

In [49]:
match2team = db.results.find({}, {'_id': 0, 'matchId': 1, 'teamId': 1})
match2team = [(x['matchId'], x['teamId']) for x in match2team]

In [84]:
for match, team in match2team:
    game_time = db.matches.find_one({'wyId': match}, {'_id': 0, 'timestamp': 1})['timestamp']
    previous_scores_json = list(db.matches.find({
        'teamsData.{}'.format(str(team)): {'$exists': True},
        'timestamp': {'$lt': game_time}
    },
    {'_id': 0, 'teamsData.{}.score'.format(str(team)): 1, 
     'teamsData.{}.scoreET'.format(str(team)): 1,
     'teamsData.{}.scoreHT'.format(str(team)): 1,
     'teamsData.{}.scoreP'.format(str(team)): 1}))
    
    scores = [x['teamsData'][str(team)]['score'] for x in previous_scores_json]
    scoresET = [x['teamsData'][str(team)]['scoreET'] for x in previous_scores_json]
    scoresHT = [x['teamsData'][str(team)]['scoreHT'] for x in previous_scores_json]
    scoresP = [x['teamsData'][str(team)]['scoreP'] for x in previous_scores_json]
    
    if scores:
        mean_score = sum(scores) / len(scores)
    else:
        mean_score = 0
    if scoresET:
        mean_scoreET = sum(scoresET) / len(scoresET)
    else:
        mean_scoreET = 0
    if scoresHT:
        mean_scoreHT = sum(scoresHT) / len(scoresHT)
    else:
        mean_scoreHT = 0
    if scoresP:
        mean_scoreP = sum(scoresP) / len(scoresP)
    else:
        mean_scoreP = 0
    
    #db.results.update_one({'matchId': match, 'teamId': team}, {'$set': {'meanPrevScore': mean_score}})
    db.results.update_one({'matchId': match, 'teamId': team}, {'$set': {'meanPrevScoreET': mean_scoreET}})
    db.results.update_one({'matchId': match, 'teamId': team}, {'$set': {'meanPrevScoreHT': mean_scoreHT}})
    db.results.update_one({'matchId': match, 'teamId': team}, {'$set': {'meanPrevScoreP': mean_scoreP}})
    